In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "1H"
max_min = 30
min_min = max_min - 15
max_sec = max_min*60
min_sec = min_min*60
table_name = "final_in_poss_df"

In [3]:
print(f'{table_name}_{half}_{min_min}_{max_min}.csv')

final_in_poss_df_1H_15_30.csv


In [4]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchPeriod == half]
df2 = df2[df2.eventSec >= min_sec]
if max_min < 45:
    df2 = df2[df2.eventSec < max_sec]

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchPeriod == half]
df3 = df3[df3.eventSec >= min_sec]
if max_min < 45:
    df3 = df3[df3.eventSec < max_sec]


with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchPeriod == half]
df4 = df4[df4.eventSec >= min_sec]
if max_min < 45:
    df4 = df4[df4.eventSec < max_sec]


with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchPeriod == half]
df5 = df5[df5.eventSec >= min_sec]
if max_min < 45:
    df5 = df5[df5.eventSec < max_sec]


with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchPeriod == half]
df6 = df6[df6.eventSec >= min_sec]
if max_min < 45:
    df6 = df6[df6.eventSec < max_sec]


with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [11]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [12]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.027778,0.138889,0.111111,0.722222
10531---2499736,0.037037,0.111111,0.111111,0.740741
10531---2499741,0.038462,0.130769,0.130769,0.700000
10531---2499757,0.042105,0.105263,0.115789,0.736842
10531---2499766,0.000000,0.068182,0.068182,0.863636
...,...,...,...,...
756---2565880,0.042254,0.126761,0.126761,0.704225
756---2565892,0.000000,0.083333,0.083333,0.833333
756---2565904,0.070175,0.192982,0.192982,0.543860


In [13]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [14]:
motif_stats

,mean,std
Motif,,
ABAB,0.035463,0.030483
ABCA,0.112169,0.038808
ABCB,0.112926,0.039024
ABCD,0.739442,0.086059


# Feature engineering - Shot DF

In [15]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-15-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [16]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-16-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [17]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-17-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [18]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [19]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499736,1,1,0,0
10531---2499777,1,0,1,0
10531---2499788,3,1,2,0
10531---2499806,2,0,1,1
10531---2499810,1,0,1,0
...,...,...,...,...
756---2565843,1,0,1,0
756---2565870,2,1,1,0
756---2565880,4,1,1,2


In [20]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [21]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499736,1,1.000000,0.000000,0.000000
10531---2499777,1,0.000000,1.000000,0.000000
10531---2499788,3,0.333333,0.666667,0.000000
10531---2499806,2,0.000000,0.500000,0.500000
10531---2499810,1,0.000000,1.000000,0.000000
...,...,...,...,...
756---2565843,1,0.000000,1.000000,0.000000
756---2565870,2,0.500000,0.500000,0.000000
756---2565880,4,0.250000,0.250000,0.500000


# Feature engineering - Pass DF - Pass Types

In [22]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-22-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [23]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [24]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
1,8,Head pass,[{'id': 1802}],265366,"[{'y': 13, 'x': 63}, {'y': 30, 'x': 94}]",2499719,Pass,1631,1H,914.585449,82,177959486,1631---2499719,2
6,8,Simple pass,[{'id': 1801}],14869,"[{'y': 17, 'x': 22}, {'y': 50, 'x': 17}]",2499719,Pass,1609,1H,933.266046,85,177959467,1609---2499719,1
7,8,Simple pass,[{'id': 1801}],3560,"[{'y': 50, 'x': 17}, {'y': 72, 'x': 30}]",2499719,Pass,1609,1H,935.800555,85,177959468,1609---2499719,1
8,8,Simple pass,[{'id': 1801}],370224,"[{'y': 72, 'x': 30}, {'y': 90, 'x': 41}]",2499719,Pass,1609,1H,939.940419,85,177959470,1609---2499719,1
9,8,Simple pass,[{'id': 1801}],167145,"[{'y': 90, 'x': 41}, {'y': 76, 'x': 31}]",2499719,Pass,1609,1H,942.417554,85,177959471,1609---2499719,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491379,8,Simple pass,[{'id': 1801}],8287,"[{'y': 59, 'x': 33}, {'y': 94, 'x': 32}]",2565927,Pass,675,1H,1788.998514,85,253301191,675---2565927,1
491380,8,Simple pass,[{'id': 1801}],4501,"[{'y': 94, 'x': 32}, {'y': 69, 'x': 38}]",2565927,Pass,675,1H,1790.385078,85,253301192,675---2565927,1
491381,8,High pass,[{'id': 1802}],3322,"[{'y': 69, 'x': 38}, {'y': 23, 'x': 66}]",2565927,Pass,675,1H,1792.067467,83,253301193,675---2565927,2
491382,8,Head pass,[{'id': 1801}],3529,"[{'y': 77, 'x': 34}, {'y': 37, 'x': 22}]",2565927,Pass,682,1H,1794.590512,82,253301235,682---2565927,2


In [25]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [26]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [27]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [28]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-28-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [29]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
1,8,Head pass,[{'id': 1802}],265366,"[{'y': 13, 'x': 63}, {'y': 30, 'x': 94}]",2499719,Pass,1631,1H,914.585449,82,177959486,1631---2499719,2,39.608080,0.350497,Forward Pass
6,8,Simple pass,[{'id': 1801}],14869,"[{'y': 17, 'x': 22}, {'y': 50, 'x': 17}]",2499719,Pass,1609,1H,933.266046,85,177959467,1609---2499719,1,27.073234,1.794273,Side Pass
7,8,Simple pass,[{'id': 1801}],3560,"[{'y': 50, 'x': 17}, {'y': 72, 'x': 30}]",2499719,Pass,1609,1H,935.800555,85,177959468,1609---2499719,1,23.518503,0.845566,Side Pass
8,8,Simple pass,[{'id': 1801}],370224,"[{'y': 72, 'x': 30}, {'y': 90, 'x': 41}]",2499719,Pass,1609,1H,939.940419,85,177959470,1609---2499719,1,19.534585,0.828849,Side Pass
9,8,Simple pass,[{'id': 1801}],167145,"[{'y': 90, 'x': 41}, {'y': 76, 'x': 31}]",2499719,Pass,1609,1H,942.417554,85,177959471,1609---2499719,2,16.414628,-2.390664,Backward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491379,8,Simple pass,[{'id': 1801}],8287,"[{'y': 59, 'x': 33}, {'y': 94, 'x': 32}]",2565927,Pass,675,1H,1788.998514,85,253301191,675---2565927,1,28.025702,1.613627,Side Pass
491380,8,Simple pass,[{'id': 1801}],4501,"[{'y': 94, 'x': 32}, {'y': 69, 'x': 38}]",2565927,Pass,675,1H,1790.385078,85,253301192,675---2565927,1,21.256528,-1.225241,Side Pass
491381,8,High pass,[{'id': 1802}],3322,"[{'y': 69, 'x': 38}, {'y': 23, 'x': 66}]",2565927,Pass,675,1H,1792.067467,83,253301193,675---2565927,2,49.831717,-0.830821,Side Pass
491382,8,Head pass,[{'id': 1801}],3529,"[{'y': 77, 'x': 34}, {'y': 37, 'x': 22}]",2565927,Pass,682,1H,1794.590512,82,253301235,682---2565927,2,35.090740,-1.993650,Side Pass


In [30]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [31]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
1,8,Head pass,[{'id': 1802}],265366,"[{'y': 13, 'x': 63}, {'y': 30, 'x': 94}]",2499719,Pass,1631,1H,914.585449,82,177959486,1631---2499719,2,39.608080,0.350497,Forward Pass
6,8,Simple pass,[{'id': 1801}],14869,"[{'y': 17, 'x': 22}, {'y': 50, 'x': 17}]",2499719,Pass,1609,1H,933.266046,85,177959467,1609---2499719,1,27.073234,1.794273,Side Pass
7,8,Simple pass,[{'id': 1801}],3560,"[{'y': 50, 'x': 17}, {'y': 72, 'x': 30}]",2499719,Pass,1609,1H,935.800555,85,177959468,1609---2499719,1,23.518503,0.845566,Side Pass
8,8,Simple pass,[{'id': 1801}],370224,"[{'y': 72, 'x': 30}, {'y': 90, 'x': 41}]",2499719,Pass,1609,1H,939.940419,85,177959470,1609---2499719,1,19.534585,0.828849,Side Pass
9,8,Simple pass,[{'id': 1801}],167145,"[{'y': 90, 'x': 41}, {'y': 76, 'x': 31}]",2499719,Pass,1609,1H,942.417554,85,177959471,1609---2499719,2,16.414628,-2.390664,Backward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491379,8,Simple pass,[{'id': 1801}],8287,"[{'y': 59, 'x': 33}, {'y': 94, 'x': 32}]",2565927,Pass,675,1H,1788.998514,85,253301191,675---2565927,1,28.025702,1.613627,Side Pass
491380,8,Simple pass,[{'id': 1801}],4501,"[{'y': 94, 'x': 32}, {'y': 69, 'x': 38}]",2565927,Pass,675,1H,1790.385078,85,253301192,675---2565927,1,21.256528,-1.225241,Side Pass
491381,8,High pass,[{'id': 1802}],3322,"[{'y': 69, 'x': 38}, {'y': 23, 'x': 66}]",2565927,Pass,675,1H,1792.067467,83,253301193,675---2565927,2,49.831717,-0.830821,Side Pass
491382,8,Head pass,[{'id': 1801}],3529,"[{'y': 77, 'x': 34}, {'y': 37, 'x': 22}]",2565927,Pass,682,1H,1794.590512,82,253301235,682---2565927,2,35.090740,-1.993650,Side Pass


In [32]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [33]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-33-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [34]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,0,5,1,8,5,0,15,4,1,6,13,20
10531---2499736,4,2,6,11,4,3,6,11,10,12,18,27
10531---2499741,1,17,2,11,21,0,28,52,1,20,32,81
10531---2499757,4,7,7,13,10,1,30,18,8,18,24,56
10531---2499766,2,6,2,13,9,0,6,5,4,10,22,15
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,5,4,5,12,7,3,22,7,9,14,22,38
756---2565892,2,7,3,6,8,0,6,15,4,12,14,25
756---2565904,3,1,2,11,7,6,19,2,9,6,24,30


## Pass Types

In [35]:
high_pass_types = ['High pass', 'Cross', 'Launch']

all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# --- overall ---
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# reindex to avoid missing columns (robust)
pass_counts['High_pass_Total'] = pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
pass_counts['Low_pass_Total']  = pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total']  = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

final_pass_types_df = pass_counts[['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']].copy()

# --- zone-specific ---
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_df = passes_df[passes_df['Zone'] == zone]

    zone_pass_counts = zone_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)

    zone_high_total = zone_pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
    zone_low_total  = zone_pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

    # avoid divide-by-zero: teams with 0 passes in that zone -> NaN
    denom = zone_pass_counts['Total_Passes'].replace(0, pd.NA)

    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_high_total / denom
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}']  = zone_low_total / denom


In [36]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3
uniqueTeamId,,,,,,,,,
10531---2499726,39,0.153846,0.846154,0.142857,0.857143,0.173913,0.826087,0.000000,1.000000
10531---2499736,57,0.210526,0.789474,0.176471,0.823529,0.285714,0.714286,0.157895,0.842105
10531---2499741,133,0.090226,0.902256,0.100000,0.888889,0.075000,0.925000,0.000000,1.000000
10531---2499757,98,0.153061,0.846939,0.142857,0.857143,0.106383,0.893617,0.312500,0.687500
10531---2499766,47,0.234043,0.765957,0.100000,0.900000,0.333333,0.666667,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...
756---2565880,74,0.216216,0.783784,0.055556,0.944444,0.230769,0.769231,0.352941,0.647059
756---2565892,51,0.176471,0.823529,0.133333,0.866667,0.214286,0.785714,0.285714,0.714286
756---2565904,60,0.283333,0.683333,0.100000,0.900000,0.272727,0.727273,0.411765,0.470588


# Pass_Connectivity

In [37]:
from collections import defaultdict
import networkx as nx

In [38]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [39]:
test_dfdf = passes_df.iloc[:1000]

In [40]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [41]:
connectivity_results

[{'1631---2499719': 1.0833333333333333, '1609---2499719': 4.822222222222222},
 {'1625---2499720': 2.8545454545454545, '1651---2499720': 0.5333333333333333},
 {'1646---2499721': 4.945454545454545, '1610---2499721': 3.066666666666667},
 {'1673---2499722': 2.7777777777777777, '1628---2499722': 2.8363636363636364},
 {'1639---2499723': 2.6944444444444446, '1623---2499723': 6.109090909090909},
 {'1611---2499724': 6.711111111111111, '1633---2499724': 1.4761904761904763},
 {'1624---2499725': 3.309090909090909, '1613---2499725': 2.0727272727272728},
 {'10531---2499726': 2.0714285714285716, '1619---2499726': 3.618181818181818},
 {'1644---2499727': 2.888888888888889, '1612---2499727': 5.781818181818182},
 {'1627---2499728': 2.8666666666666667, '1659---2499728': 4.222222222222222},
 {'1659---2499729': 3.977777777777778, '1644---2499729': 4.0},
 {'1627---2499730': 2.25, '1646---2499730': 4.709090909090909},
 {'1613---2499731': 4.555555555555555, '1673---2499731': 2.8727272727272726},
 {'1651---2499

In [42]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [43]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1631---2499719,1.083333
1609---2499719,4.822222
1625---2499720,2.854545
1651---2499720,0.533333
1646---2499721,4.945455
...,...
3158---2576336,3.288889
3164---2576337,4.218182
3204---2576337,2.388889


In [44]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [45]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.027778,0.138889,0.111111,0.722222,39,0.153846,0.846154,0.142857,0.857143,0.173913,...,0.102564,0.025641,0.153846,0.333333,0.512821,NaN,NaN,NaN,NaN,2.071429
10531---2499736,0.037037,0.111111,0.111111,0.740741,57,0.210526,0.789474,0.176471,0.823529,0.285714,...,0.192982,0.175439,0.210526,0.315789,0.473684,1.0,1.00,0.000000,0.000000,3.527273
10531---2499741,0.038462,0.130769,0.130769,0.700000,133,0.090226,0.902256,0.100000,0.888889,0.075000,...,0.390977,0.007519,0.150376,0.240602,0.609023,NaN,NaN,NaN,NaN,4.072727
10531---2499757,0.042105,0.105263,0.115789,0.736842,98,0.153061,0.846939,0.142857,0.857143,0.106383,...,0.183673,0.081633,0.183673,0.244898,0.571429,NaN,NaN,NaN,NaN,4.955556
10531---2499766,0.000000,0.068182,0.068182,0.863636,47,0.234043,0.765957,0.100000,0.900000,0.333333,...,0.106383,0.085106,0.212766,0.468085,0.319149,NaN,NaN,NaN,NaN,3.618182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.042254,0.126761,0.126761,0.704225,74,0.216216,0.783784,0.055556,0.944444,0.230769,...,0.094595,0.121622,0.189189,0.297297,0.513514,4.0,0.25,0.250000,0.500000,3.327273
756---2565892,0.000000,0.083333,0.083333,0.833333,51,0.176471,0.823529,0.133333,0.866667,0.214286,...,0.294118,0.078431,0.235294,0.274510,0.490196,NaN,NaN,NaN,NaN,3.377778
756---2565904,0.070175,0.192982,0.192982,0.543860,60,0.283333,0.683333,0.100000,0.900000,0.272727,...,0.033333,0.150000,0.100000,0.400000,0.500000,3.0,0.00,0.333333,0.666667,4.044444


In [46]:

if half is not None:
    final_df.to_csv(f'{table_name}_{half}_{min_min}_{max_min}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [47]:
half

'1H'

# END OF FILE